# Project Start Here

In [107]:
import requests as re
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

## get the data
---
- Get the data by api.
- Save the data as 'raw.csv', which is the raw data ready for data preporcessing.
---
Totaly 2,000 pics of data sorted by time from resent to past.

In [108]:
def get_data():
    zed_index = '?'
    include_sold = '1'
    page_number = 1
    listing_status = 'sale'
    page_size = '100'
    order_by = 'age'
    area = 'somerset'
    api_key = 'jwttfkz79asz2d9h8sms7tuu'

    url = f"http://api.zoopla.co.uk/api/v1/property_listings.js?zed_index=?&include_sold=1&page_number={page_number}&listing_status=sale&page_size=100&order_by=age&area=somerset&api_key=jwttfkz79asz2d9h8sms7tuu"
    for page_number in range(1, 59):
        response = re.get(url)
        df = json_normalize(response.json()['listing'])
        if page_number == 1:
            df.to_csv('raw.csv', index = False)
        else:
            df.to_csv('raw.csv', index = False, mode = 'a', header=False)

In [ ]:
## Load Raw Data
get_data()

In [49]:
raw = pd.read_csv('raw.csv')

In [50]:
raw

,agent_address,agent_logo,agent_name,agent_phone,category,country,country_code,county,description,details_url,...,outcode,post_town,price,price_change,price_modifier,property_type,short_description,status,street_name,thumbnail_url
0,"Nationwide Estate Agent, Head Office: Suite 7,...",https://st.zoocdn.com/zoopla_static_agent_logo...,"Purplebricks, Head Office",024 7511 8874,Residential,England,gb,Somerset,"A spacious ground floor flat, available with n...",https://www.zoopla.co.uk/for-sale/details/5178...,...,TA2,Taunton,120000,"[{'direction': '', 'date': '2019-06-16 23:05:0...",NaN,Flat,"A spacious ground floor flat, available with n...",for_sale,Portman Street,https://lid.zoocdn.com/80/60/de5349f165b325e74...
1,"14 Gannet Road, Worle, Weston Super Mare",https://st.zoocdn.com/zoopla_static_agent_logo...,House Fox,01934 282950,Residential,England,gb,Somerset,A good size detached house set on this sought ...,https://www.zoopla.co.uk/for-sale/details/5178...,...,BS27,Cheddar,370000,"[{'direction': '', 'date': '2019-06-16 00:01:1...",NaN,Detached house,A good size detached house set on this sought ...,for_sale,Charterhouse Close,https://lid.zoocdn.com/80/60/4f241ccb65011b0ca...
2,"Nationwide Estate Agent, Head Office: Suite 7,...",https://st.zoocdn.com/zoopla_static_agent_logo...,"Purplebricks, Head Office",024 7511 8874,Residential,England,gb,Somerset,This is a lovely spacious 3 bedroom terraced h...,https://www.zoopla.co.uk/for-sale/details/5178...,...,TA21,Wellington,200000,"[{'direction': '', 'date': '2019-06-15 19:26:4...",NaN,Terraced house,This is a lovely spacious 3 bedroom terraced h...,for_sale,Drakes Park,https://lid.zoocdn.com/80/60/4f2b34005a6cc33b1...
3,"110 High Street, Worle, Weston-Super-Mare",https://st.zoocdn.com/zoopla_static_agent_logo...,Cooke & Co,01934 247816,Residential,England,gb,North Somerset,One off design Detached Bungalow with accommod...,https://www.zoopla.co.uk/for-sale/details/5178...,...,BS22,Weston-super-Mare,219950,"[{'direction': '', 'date': '2019-06-15 16:43:3...",NaN,Detached bungalow,One off design Detached Bungalow with accommod...,for_sale,Whittington Drive,https://lid.zoocdn.com/80/60/ae1c91fa8b722407a...
4,"30 Woodborough Road, Winscombe",https://st.zoocdn.com/zoopla_static_agent_logo...,Farrons Estate Agents,01934 247089,Residential,England,gb,North Somerset,Individual Detached HouseApprox 1/2 Acre of Gr...,https://www.zoopla.co.uk/for-sale/details/5178...,...,BS25,Winscombe,659000,"[{'direction': '', 'date': '2019-06-15 17:30:2...",NaN,Detached house,5 reception rooms. 4 double bedrooms &amp; 2 e...,for_sale,Horseleaze Lane,https://lid.zoocdn.com/80/60/db04ae945d8aa38b8...
5,"35 Fore Street, Chard",https://st.zoocdn.com/zoopla_static_agent_logo...,Tarr Residential,01460 247031,Residential,England,gb,Somerset,An extremely well presented detached 4 bedroom...,https://www.zoopla.co.uk/for-sale/details/5178...,...,TA20,Chard,350000,"[{'direction': '', 'date': '2019-06-15 17:14:0...",guide_price,Detached house,"<p class=""top"">An extremely well presented det...",for_sale,Caraway Close,https://lid.zoocdn.com/80/60/8c9410f8613c513cd...
6,"Park 5 Park Road, Yeovil",https://st.zoocdn.com/zoopla_static_agent_logo...,Evolve,01935 590148,Residential,England,gb,Somerset,Detailed DescriptionEvolve estate agents offer...,https://www.zoopla.co.uk/for-sale/details/5178...,...,BA21,Yeovil,160000,"[{'direction': '', 'date': '2019-06-15 17:05:2...",offers_over,Terraced house,Evolve estate agents offer for sale this well ...,for_sale,Grass Royal,https://lid.zoocdn.com/80/60/39ca56ff735989d1d...
7,"Park 5 Park Road, Yeovil",https://st.zoocdn.com/zoopla_static_agent_logo...,Evolve,01935 590148,Residential,England,gb,Somerset,Detailed DescriptionPositioned in the heart of...,https://www.zoopla.co.uk/for-sale/details/5178...,...,BA20,Yeovil,150000,"[{'direction': '', 'date': '2019-06-15 17:05:2...",NaN,Flat,Positioned in the heart of town and located in...,for_sale,Flowers House,https://lid.zoocdn.com/80/60

### For get data testing
---
Testing the api and csv save method

In [45]:
zed_index = '?'
include_sold = '1'
page_number = 3
listing_status = 'sale'
page_size = '20'
order_by = 'age'
area = 'somerset'
api_key = 'jwttfkz79asz2d9h8sms7tuu'

url = f'http://api.zoopla.co.uk/api/v1/property_listings.js?zed_index=?&include_sold=1&page_number={page_number}&listing_status=sale&page_size=10&order_by=age&area=somerset&api_key=jwttfkz79asz2d9h8sms7tuu'
response = re.get(url)
df = json_normalize(response.json()['listing'])
df.to_csv('raw.csv', index = False)

In [ ]:
df.head()

# Data Preporcessing

In [51]:
raw.columns

Index(['agent_address', 'agent_logo', 'agent_name', 'agent_phone', 'category',
       'country', 'country_code', 'county', 'description', 'details_url',
       'displayable_address', 'first_published_date', 'floor_plan',
       'furnished_state', 'image_150_113_url', 'image_354_255_url',
       'image_50_38_url', 'image_645_430_url', 'image_80_60_url',
       'image_caption', 'image_url', 'last_published_date', 'latitude',
       'letting_fees', 'listing_id', 'listing_status',
       'location_is_approximate', 'longitude', 'num_bathrooms', 'num_bedrooms',
       'num_floors', 'num_recepts', 'outcode', 'post_town', 'price',
       'price_change', 'price_modifier', 'property_type', 'short_description',
       'status', 'street_name', 'thumbnail_url'],
      dtype='object')

In [98]:
useful_information = raw[['property_type','category', 'description', 'num_bathrooms', 'num_bedrooms','num_floors', 'num_recepts', 'outcode', 'post_town', 'street_name', 'price']].reset_index()

In [99]:
cate = useful_information.drop_duplicates(['category'])
cate

,index,property_type,category,description,num_bathrooms,num_bedrooms,num_floors,num_recepts,outcode,post_town,street_name,price
0,0,Flat,Residential,"A spacious ground floor flat, available with n...",1,1,0,1,TA2,Taunton,Portman Street,120000


In [100]:
property_type = useful_information.drop_duplicates(['property_type'])
property_type['property_type']

0                   Flat
1         Detached house
2         Terraced house
3      Detached bungalow
8      End terrace house
9    Semi-detached house
Name: property_type, dtype: object

In [101]:
code = useful_information.drop_duplicates(['outcode'])
code['outcode']

0     TA2
1    BS27
2    TA21
3    BS22
4    BS25
5    TA20
6    BA21
7    BA20
Name: outcode, dtype: object

In [102]:
post_town = useful_information.drop_duplicates(['post_town'])
post_town['post_town']

0              Taunton
1              Cheddar
2           Wellington
3    Weston-super-Mare
4            Winscombe
5                Chard
6               Yeovil
Name: post_town, dtype: object

In [103]:
street_name = useful_information.drop_duplicates(['street_name'])
street_name['street_name']

0        Portman Street
1    Charterhouse Close
2           Drakes Park
3     Whittington Drive
4       Horseleaze Lane
5         Caraway Close
6           Grass Royal
7         Flowers House
8            Great Mead
9         Buckland Road
Name: street_name, dtype: object

In [105]:
## one-hot encode
useful_information = pd.get_dummies(useful_information)

In [106]:
useful_information

,index,num_bathrooms,num_bedrooms,num_floors,num_recepts,price,property_type_Detached bungalow,property_type_Detached house,property_type_End terrace house,property_type_Flat,...,street_name_Buckland Road,street_name_Caraway Close,street_name_Charterhouse Close,street_name_Drakes Park,street_name_Flowers House,street_name_Grass Royal,street_name_Great Mead,street_name_Horseleaze Lane,street_name_Portman Street,street_name_Whittington Drive
0,0,1,1,0,1,120000,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,1,0,4,0,0,370000,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2,1,3,0,2,200000,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,0,2,0,0,219950,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,0,4,0,0,659000,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,5,2,4,0,2,350000,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
6,6,1,3,0,2,160000,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,7,1,3,0,1,150000,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
8,8,2,2,0,1,170000,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
9,9,1,2,0,1,169950,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
